In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

website = "https://yokohama-atg.com/usa/tire-selector-yokohama-off-highway-tires/"
path = "C:\\Users\\cheta\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"  # Change according to your PC's file path
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

my_designs = ['350']

folder = 'Alliance'

if not os.path.exists(folder):
    os.mkdir(folder)

try:
    driver.get(website)

    # Wait for the cookie accept button and click it
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@data-cookie-set="accept"]'))
        )
        accept_button.click()
    except Exception as e:
        print(f"Cookie notice not found or could not be clicked: {str(e)}")

    print('working...')
    
    # Click on the brand select button
    select_buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="select2-arrow"]'))
    )
    brands_button = select_buttons[2]
    brands_button.click()

    # Select the brand
    select_brands = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]'))
    )
    select_brands[0].click()

    # Wait for the loader to disappear
    WebDriverWait(driver, 10).until(
        EC.invisibility_of_element((By.XPATH, '//div[@class="loader-wrapper"]'))
    )

    # Click on the design select button
    select_buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="select2-arrow"]'))
    )
    design_button = select_buttons[3]
    design_button.click()

    select_design = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]'))
    )

    def get_design(values):
        new_design = []

        if isinstance(values, str):
            values = [values]

        for des in select_design:
            if des.text in values:
                new_design.append(des)
        return new_design

    designs = get_design(my_designs)
    design = designs[0]

    print(f'For Design: {design.text}')
    design.click()

    # Click the search button
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@title="Search"]'))
    )
    search_button.click()
    print('Search...')

    # Wait for tables to become visible
    try:
        # Find the parent div element containing the tables you're interested in
        data_in_mm = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="data_in_mm "]'))
        )

        # Find all tables within the parent div
        tables_mm = data_in_mm.find_elements(By.XPATH, './/table[@id="tblResultView"]')
        headers = []
        data = []
        table_headers_in_mm = []
        table_data_mm = []

        for i, table in enumerate(tables_mm):
            # Extract header row
            header_row_mm = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'tr')
            headers_heads = {index: heads.find_elements(By.TAG_NAME, 'th') for index, heads in enumerate(header_row_mm) if index!=3}

            headers_heads_texts = {
                index: {i: th.text for i, th in enumerate(th_list)}
                for index, th_list in headers_heads.items()
            }

            table_headers_in_mm.append({f"Table_{i+1}": headers_heads_texts})
            print('Header Collected')
        
        headers.append({"Tables_in_mm": table_headers_in_mm})



        # Testing

        # table_data = tables_mm[0].find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
        # print([dt.text for dt in table_data])



        for i, table in enumerate(tables_mm):
            table_data_in_mm = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
            data_rows_mm = {index: row.find_elements(By.TAG_NAME, 'td') for index, row in enumerate(table_data_in_mm)}

            data_rows_texts_mm = {
                index: {i: td.text for i, td in enumerate(td_list)}
                for index, td_list in data_rows_mm.items()
            }

            table_data_mm.append({f"Table_{i+1}": data_rows_texts_mm})
            print('Table Data Collected')

        data.append({"Table_data_mm": table_data_mm})


        data_in_inch = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="data_in_inch "]'))
        )

        # Find all table elements within the parent div
        tables_inch = data_in_inch.find_elements(By.XPATH, './/table[@id="tblResultView"]')

        # Execute JavaScript to change the display style to block (making it visible)
        driver.execute_script("arguments[0].style.display = 'block';", data_in_inch)

        table_headers_in_inch = []
        table_data_inch = []


        for i, table in enumerate(tables_inch):
            # Extract header row
            header_row_inch = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'tr')
            headers_heads = {index: heads.find_elements(By.TAG_NAME, 'th') for index, heads in enumerate(header_row_inch) if index!=3}

            headers_heads_texts = {
                index: {i: th.text for i, th in enumerate(th_list)}
                for index, th_list in headers_heads.items()
            }

            table_headers_in_inch.append({f"Table_{i+1}": headers_heads_texts})
            print('Header Collected')
        
        headers.append({"Tables_in_inch": table_headers_in_inch})

        for i, table in enumerate(tables_inch):
            table_data_in_inch = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
            data_rows_inch = {index: row.find_elements(By.TAG_NAME, 'td') for index, row in enumerate(table_data_in_inch)}

            data_rows_texts_inch = {
                index: {i: td.text for i, td in enumerate(td_list)}
                for index, td_list in data_rows_inch.items()
            }

            table_data_inch.append({f"Table_{i+1}": data_rows_texts_inch})
            print('Table Data Collected')

        data.append({"Table_data_inch": table_data_inch})

    except Exception as e:
        print(f"An error occurred while fetching tables 'tblResultView': {str(e)}")

finally:
    time.sleep(5)
    driver.quit()
    print("All data has been collected and the browser is closed.")

working...
For Design: 350
Search...
Header Collected
Header Collected
Table Data Collected
Table Data Collected
Header Collected
Header Collected
Table Data Collected
Table Data Collected
All data has been collected and the browser is closed.


In [32]:
data

[{'Table_data_mm': [{'Table_1': {0: {0: '300/95R52 (12.4R52)',
      1: 'W10X52 W9X52',
      2: '310',
      3: '1890',
      4: '887',
      5: '5781',
      6: '156D\n159A8',
      7: '0.8',
      8: '2100',
      9: '1720',
      10: '1660',
      11: '1610',
      12: '1530',
      13: '1470',
      14: '1400'},
     1: {0: '1.2',
      1: '2700',
      2: '2210',
      3: '2130',
      4: '2070',
      5: '1970',
      6: '1890',
      7: '1800'},
     2: {0: '1.6',
      1: '3270',
      2: '2680',
      3: '2580',
      4: '2510',
      5: '2390',
      6: '2290',
      7: '2180'},
     3: {0: '2',
      1: '3650',
      2: '2990',
      3: '2880',
      4: '2790',
      5: '2660',
      6: '2550',
      7: '2430'},
     4: {0: '2.4',
      1: '3860',
      2: '3170',
      3: '3050',
      4: '2960',
      5: '2820',
      6: '2700',
      7: '2575'},
     5: {0: '2.8',
      1: '4200',
      2: '3440',
      3: '3320',
      4: '3220',
      5: '3070',
      6: '2940',
      

In [43]:
headers

[{'Tables_in_mm': [{'Table_1': {0: {0: 'Size',
      1: 'Rim',
      2: 'Unloaded Dimension',
      3: 'Loaded Static radius\nmm',
      4: 'Rolling Circumference\nmm',
      5: 'Load Index\nPR\nSymbols',
      6: 'Inflation Pressure\nbar',
      7: 'Recommended Load'},
     1: {0: 'Speed'},
     2: {0: 'Not high and sustained torque;Road transport'},
     4: {0: 'SW\nmm',
      1: 'OD\nmm',
      2: '10\nkmph',
      3: '20\nkmph',
      4: '25\nkmph',
      5: '30\nkmph',
      6: '40\nkmph',
      7: '50\nkmph',
      8: '65\nkmph'}}},
   {'Table_2': {0: {0: 'Size',
      1: 'Rim',
      2: 'Unloaded Dimension',
      3: 'Loaded Static radius\nmm',
      4: 'Rolling Circumference\nmm',
      5: 'Load Index\nPR\nSymbols',
      6: 'Inflation Pressure\nbar',
      7: 'Recommended Load'},
     1: {0: 'Speed'},
     2: {0: 'CYCLIC OPERATION'},
     4: {0: 'SW\nmm', 1: 'OD\nmm', 2: '10\nkmph', 3: '15\nkmph'}}}]},
 {'Tables_in_inch': [{'Table_1': {0: {0: 'Size',
      1: 'Rim',
      2: '

In [40]:
import pandas as pd

In [53]:
table1_mm = headers[0]['Tables_in_mm'][0]['Table_1']
table2_mm = headers[0]['Tables_in_mm'][1]['Table_2']
table1_inch = headers[1]['Tables_in_inch'][0]['Table_1']
table2_inch = headers[1]['Tables_in_inch'][1]['Table_2']


In [55]:
print(table1_mm)
print(table2_mm)

{0: {0: 'Size', 1: 'Rim', 2: 'Unloaded Dimension', 3: 'Loaded Static radius\nmm', 4: 'Rolling Circumference\nmm', 5: 'Load Index\nPR\nSymbols', 6: 'Inflation Pressure\nbar', 7: 'Recommended Load'}, 1: {0: 'Speed'}, 2: {0: 'Not high and sustained torque;Road transport'}, 4: {0: 'SW\nmm', 1: 'OD\nmm', 2: '10\nkmph', 3: '20\nkmph', 4: '25\nkmph', 5: '30\nkmph', 6: '40\nkmph', 7: '50\nkmph', 8: '65\nkmph'}}
{0: {0: 'Size', 1: 'Rim', 2: 'Unloaded Dimension', 3: 'Loaded Static radius\nmm', 4: 'Rolling Circumference\nmm', 5: 'Load Index\nPR\nSymbols', 6: 'Inflation Pressure\nbar', 7: 'Recommended Load'}, 1: {0: 'Speed'}, 2: {0: 'CYCLIC OPERATION'}, 4: {0: 'SW\nmm', 1: 'OD\nmm', 2: '10\nkmph', 3: '15\nkmph'}}


In [48]:
table1_inch

{0: {0: 'Size',
  1: 'Rim',
  2: 'Unloaded Dimension',
  3: 'Loaded Static radius\ninch',
  4: 'Rolling Circumference\ninch',
  5: 'Load Index\nPR\nSymbols',
  6: 'Inflation Pressure\npsi',
  7: 'Recommended Load'},
 1: {0: 'Speed'},
 2: {0: 'Not high and sustained torque;Road transport'},
 4: {0: 'SW\ninch',
  1: 'OD\ninch',
  2: '6\nmph',
  3: '12\nmph',
  4: '16\nmph',
  5: '19\nmph',
  6: '25\nmph',
  7: '31\nmph',
  8: '40\nmph'}}

In [15]:
import pandas as pd

pd.DataFrame(columns=list(table[0].values()))

,Size,Rim,Unloaded Dimension,Loaded Static radius\nmm,Rolling Circumference\nmm,Load Index\nPR\nSymbols,Inflation Pressure\nbar,Recommended Load


In [59]:
row1 = list(table2_mm[0].values())
row2 = list(table2_mm[1].values())
row3 = list(table2_mm[2].values())
row4 = list(table2_mm[4].values())

row1 = [text.replace('\n', ' ') for text in row1]
print('Row 1:', row1)

print('Row 2:', row2)

print('Row 3:', row3)

row4 = [text.replace('\n', ' ') for text in row4]
print('Row 4:', row4)

Row 1: ['Size', 'Rim', 'Unloaded Dimension', 'Loaded Static radius mm', 'Rolling Circumference mm', 'Load Index PR Symbols', 'Inflation Pressure bar', 'Recommended Load']
Row 2: ['Speed']
Row 3: ['CYCLIC OPERATION']
Row 4: ['SW mm', 'OD mm', '10 kmph', '15 kmph']


In [57]:
row1 = list(table1_inch[0].values())
row2 = list(table1_inch[1].values())
row3 = list(table1_inch[2].values())
row4 = list(table1_inch[4].values())

row1 = [text.replace('\n', ' ') for text in row1]
print('Row 1:', row1)

print('Row 2:', row2)

print('Row 3:', row3)

row4 = [text.replace('\n', ' ') for text in row4]
print('Row 4:', row4)

Row 1: ['Size', 'Rim', 'Unloaded Dimension', 'Loaded Static radius inch', 'Rolling Circumference inch', 'Load Index PR Symbols', 'Inflation Pressure psi', 'Recommended Load']
Row 2: ['Speed']
Row 3: ['Not high and sustained torque;Road transport']
Row 4: ['SW inch', 'OD inch', '6 mph', '12 mph', '16 mph', '19 mph', '25 mph', '31 mph', '40 mph']


In [61]:
tuples = []

for col in row1:
    if col == 'Unloaded Dimension':
        tuples.append((col, '', '', row4[0]))
        tuples.append((col, '', '', row4[1]))
    elif col == 'Recommended Load':
        for subcol in row4[2:]:
            tuples.append((col, row2[0], row3[0], subcol))
    else:
        tuples.append((col, '', '', ''))
tuples

sample_data = [
    ['245/70R19.5', '19.5x7.50', 410, 2580, '134/132 M', 8.5, 'S', 'M', 2720, 2550],
    ['265/70R19.5', '19.5x7.50', 420, 2650, '136/134 M', 8.5, 'S', 'M', 2900, 2700]
]
columns = pd.MultiIndex.from_tuples(tuples)
df = pd.DataFrame(sample_data, columns=columns)
df

Size        Rim Unloaded Dimension       Loaded Static radius mm  \
                                                                             
                                                                             
                                       SW mm OD mm                           
0  245/70R19.5  19.5x7.50                410  2580               134/132 M   
1  265/70R19.5  19.5x7.50                420  2650               136/134 M   

  Rolling Circumference mm Load Index PR Symbols Inflation Pressure bar  \
                                                                          
                                                                          
                                                                          
0                      8.5                     S                      M   
1                      8.5                     S                      M   

  Recommended Load          
             Speed          
  CYCLIC OPERATION          
           10 kmph 15 kmph  
0             2720    2550  
1             2900    2700

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

website = "https://yokohama-atg.com/usa/tire-selector-yokohama-off-highway-tires/"
path = "C:\\Users\\cheta\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"  # Change according to your PC's file path
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

my_designs = ['350']
folder = 'Alliance'

if not os.path.exists(folder):
    os.mkdir(folder)

def wait_and_click(driver, by, value, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((by, value))
        )
        element.click()
    except Exception as e:
        print(f"Element not found or could not be clicked: {str(e)}")

def get_select_elements(driver, by, value, timeout=10):
    try:
        elements = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((by, value))
        )
        return elements
    except Exception as e:
        print(f"Elements not found: {str(e)}")
        return []

def get_designs(designs_list, available_designs):
    selected_designs = []
    for design in available_designs:
        if design.text in designs_list:
            selected_designs.append(design)
    return selected_designs

def collect_table_headers(tables):
    table_data = []
    for i, table in enumerate(tables):
        header_rows = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'tr')
        headers_dict = {index: row.find_elements(By.TAG_NAME, 'th') for index, row in enumerate(header_rows) if index != 3}
        headers_text = {
            index: {i: th.text for i, th in enumerate(th_list)}
            for index, th_list in headers_dict.items()
        }
        table_data.append({f"Table_{i+1}": headers_text})
    return table_data

def collect_data(driver, div_class):
    try:
        data_div = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, f'//div[@class="{div_class} "]'))
        )
        tables = data_div.find_elements(By.XPATH, './/table[@id="tblResultView"]')
        if div_class == 'data_in_inch':
            driver.execute_script("arguments[0].style.display = 'block';", data_div)
        return collect_table_headers(tables)
    except Exception as e:
        print(f"An error occurred while fetching tables: {str(e)}")
        return []

try:
    driver.get(website)

    # Accept cookies
    wait_and_click(driver, By.XPATH, '//span[@data-cookie-set="accept"]')

    # Select brand
    select_buttons = get_select_elements(driver, By.XPATH, '//span[@class="select2-arrow"]')
    brands_button = select_buttons[2]
    brands_button.click()
    brands = get_select_elements(driver, By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]')
    brands[0].click()

    # Wait for the loader to disappear
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.XPATH, '//div[@class="loader-wrapper"]')))

    # Select design
    design_button = select_buttons[3]
    design_button.click()
    available_designs = get_select_elements(driver, By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]')
    selected_designs = get_designs(my_designs, available_designs)
    if selected_designs:
        selected_designs[0].click()

    # Click search button
    wait_and_click(driver, By.XPATH, '//a[@title="Search"]')

    # Collect table headers
    headers = {
        "Tables_in_mm": collect_data(driver, "data_in_mm"),
        "Tables_in_inch": collect_data(driver, "data_in_inch")
    }
    print('*'*55)
    print()
    print("Headers Collected:", list(headers.keys()))
    print()
    print('*'*55)

finally:
    time.sleep(5)
    driver.quit()
    print("All data has been collected and the browser is closed.")


*******************************************************

Headers Collected: ['Tables_in_mm', 'Tables_in_inch']

*******************************************************
All data has been collected and the browser is closed.


In [10]:
import pandas as pd

def extract_headers(table):
    """
    Extracts and processes the headers from the table.
    """
    row1 = [text.replace('\n', ' ') for text in list(table[0].values())]
    row2 = list(table[1].values())
    row3 = list(table[2].values())
    row4 = [text.replace('\n', ' ') for text in list(table[4].values())]
    
    return row1, row2, row3, row4

def create_multi_index(row1, row2, row3, row4):
    """
    Creates a multi-level index for DataFrame columns based on the headers.
    """
    tuples = []
    for col in row1:
        if col == 'Unloaded Dimension':
            tuples.extend([(col, '', '', row4[0]), (col, '', '', row4[1])])
        elif col == 'Recommended Load':
            tuples.extend([(col, row2[0], row3[0], subcol) for subcol in row4[2:]])
        else:
            tuples.append((col, '', '', ''))
    
    return pd.MultiIndex.from_tuples(tuples)

def create_dataframe(headers, sample_data):
    """
    Creates a DataFrame from the processed headers and sample data.
    """
    row1, row2, row3, row4 = extract_headers(headers)
    columns = create_multi_index(row1, row2, row3, row4)
    return pd.DataFrame(sample_data, columns=columns)

def save_to_csv(df, filename):
    """
    Saves the DataFrame to a CSV file.
    """
    df.to_csv(filename, index=False)
    print(f"CSV file '{filename}' saved successfully.")

table1_sample_data = [
    ['245/70R19.5', '19.5x7.50', 410, 2580, '134/132 M', 8.5, 'S', 'M', 2720, 2550, 2720, 2720, 2550, 2550, 2550],
    ['265/70R19.5', '19.5x7.50', 420, 2650, '136/134 M', 8.5, 'S', 'M', 2900, 2700, 2550, 2720, 2720, 2550, 2550]
]

table2_sample_data = [
    ['245/70R19.5', '19.5x7.50', 410, 2580, '134/132 M', 8.5, 'S', 'M', 2720, 2550],
    ['265/70R19.5', '19.5x7.50', 420, 2650, '136/134 M', 8.5, 'S', 'M', 2900, 2700]
]

# Process and save each table
df_table1_mm = create_dataframe(headers['Tables_in_mm'][0]['Table_1'], table1_sample_data)
save_to_csv(df_table1_mm, 'table1_mm.csv')

df_table2_mm = create_dataframe(headers['Tables_in_mm'][1]['Table_2'], table2_sample_data)
save_to_csv(df_table2_mm, 'table2_mm.csv')

df_table1_inch = create_dataframe(headers['Tables_in_inch'][0]['Table_1'], table1_sample_data)
save_to_csv(df_table1_inch, 'table1_inch.csv')

df_table2_inch = create_dataframe(headers['Tables_in_inch'][1]['Table_2'], table2_sample_data)
save_to_csv(df_table2_inch, 'table2_inch.csv')


CSV file 'table1_mm.csv' saved successfully.
CSV file 'table2_mm.csv' saved successfully.
CSV file 'table1_inch.csv' saved successfully.
CSV file 'table2_inch.csv' saved successfully.


In [150]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

def setup_driver(driver_path):
    """
    Set up the Chrome WebDriver.
    """
    service = Service(executable_path=driver_path)
    return webdriver.Chrome(service=service)

def wait_and_click(driver, by, value, timeout=10):
    """
    Wait for an element to be clickable and click it.
    """
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((by, value))
        )
        element.click()
    except Exception as e:
        print(f"Element not found or could not be clicked: {str(e)}")

def get_select_elements(driver, by, value, timeout=10):
    """
    Wait for and return all elements matching the selector.
    """
    try:
        elements = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((by, value))
        )
        return elements
    except Exception as e:
        print(f"Elements not found: {str(e)}")
        return []

def get_designs(designs_list, available_designs):
    """
    Filter and return the selected designs from the available designs list.
    """
    selected_designs = []
    for design in available_designs:
        if design.text in designs_list:
            selected_designs.append(design)
    return selected_designs

def collect_table_info(driver, div_class):
    """
    Collect both table headers and data based on the specified div class.
    """
    try:
        data_div = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, f'//div[@class="{div_class} "]'))
        )
        tables = data_div.find_elements(By.XPATH, './/table[@id="tblResultView"]')
        if div_class == 'data_in_inch':
            driver.execute_script("arguments[0].style.display = 'block';", data_div)
        
        table_info = {}
        for i, table in enumerate(tables):
            header_rows = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'tr')
            headers_dict = {index: row.find_elements(By.TAG_NAME, 'th') for index, row in enumerate(header_rows) if index != 3}
            headers_text = {
                index: {i: th.text for i, th in enumerate(th_list)}
                for index, th_list in headers_dict.items()
            }
            
            table_data_rows = table.find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME, 'tr')
            data_rows = [{i: td.text for i, td in enumerate(row.find_elements(By.TAG_NAME, 'td'))} for row in table_data_rows]
            
            table_info[f"Table_{i+1}"] = {
                    "headers": headers_text,
                    "data": data_rows
                }
        
        return table_info
    
    except Exception as e:
        print(f"An error occurred while fetching tables: {str(e)}")
        return []

# Configuration
website = "https://yokohama-atg.com/usa/tire-selector-yokohama-off-highway-tires/"
driver_path = "C:\\Users\\cheta\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"  # Change according to your PC's file path
my_designs = ['350']
folder = 'Alliance'

if not os.path.exists(folder):
    os.mkdir(folder)

try:
    # Initialize WebDriver
    driver = setup_driver(driver_path)
    driver.get(website)

    # Accept cookies
    wait_and_click(driver, By.XPATH, '//span[@data-cookie-set="accept"]')

    # Select brand
    select_buttons = get_select_elements(driver, By.XPATH, '//span[@class="select2-arrow"]')
    brands_button = select_buttons[2]
    brands_button.click()
    brands = get_select_elements(driver, By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]')
    brands[0].click()

    # Wait for the loader to disappear
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.XPATH, '//div[@class="loader-wrapper"]')))

    # Select design
    design_button = select_buttons[3]
    design_button.click()
    available_designs = get_select_elements(driver, By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]')
    selected_designs = get_designs(my_designs, available_designs)
    if selected_designs:
        selected_designs[0].click()

    # Click search button
    wait_and_click(driver, By.XPATH, '//a[@title="Search"]')

    # Collect data for both mm and inch tables
    international_table_info = collect_table_info(driver, "data_in_mm")
    usa_table_info = collect_table_info(driver, "data_in_inch")

    # Print collected data headers
    print('*'*55)
    print()
    print("Headers Collected for mm tables")
    print("Data Collected for mm tables")
    print()
    print("Headers Collected for inch tables")
    print("Data Collected for inch tables")
    print()
    print('*'*55)

finally:
    time.sleep(5)
    driver.quit()
    print("All data has been collected and the browser is closed.")


*******************************************************

Headers Collected for mm tables
Data Collected for mm tables

Headers Collected for inch tables
Data Collected for inch tables

*******************************************************
All data has been collected and the browser is closed.


In [151]:
international_table_info

{'Table_1': {'headers': {0: {0: 'Size',
    1: 'Rim',
    2: 'Unloaded Dimension',
    3: 'Loaded Static radius\nmm',
    4: 'Rolling Circumference\nmm',
    5: 'Load Index\nPR\nSymbols',
    6: 'Inflation Pressure\nbar',
    7: 'Recommended Load'},
   1: {0: 'Speed'},
   2: {0: 'Not high and sustained torque;Road transport'},
   4: {0: 'SW\nmm',
    1: 'OD\nmm',
    2: '10\nkmph',
    3: '20\nkmph',
    4: '25\nkmph',
    5: '30\nkmph',
    6: '40\nkmph',
    7: '50\nkmph',
    8: '65\nkmph'}},
  'data': [{0: '300/95R52 (12.4R52)',
    1: 'W10X52 W9X52',
    2: '310',
    3: '1890',
    4: '887',
    5: '5781',
    6: '156D\n159A8',
    7: '0.8',
    8: '2100',
    9: '1720',
    10: '1660',
    11: '1610',
    12: '1530',
    13: '1470',
    14: '1400'},
   {0: '1.2',
    1: '2700',
    2: '2210',
    3: '2130',
    4: '2070',
    5: '1970',
    6: '1890',
    7: '1800'},
   {0: '1.6',
    1: '3270',
    2: '2680',
    3: '2580',
    4: '2510',
    5: '2390',
    6: '2290',
    7: '2

In [163]:
import pandas as pd

def extract_headers(table):
    """
    Extracts and processes the headers from the table.
    """
    row1 = [text.replace('\n', ' ') for text in list(table[0].values())]
    row2 = list(table[1].values())
    row3 = list(table[2].values())
    row4 = [text.replace('\n', ' ') for text in list(table[4].values())]
    
    return row1, row2, row3, row4

def create_multi_index(row1, row2, row3, row4):
    """
    Creates a multi-level index for DataFrame columns based on the headers.
    """
    tuples = []
    for col in row1:
        if col == 'Unloaded Dimension':
            tuples.extend([(col, '', '', row4[0]), (col, '', '', row4[1])])
        elif col == 'Recommended Load':
            tuples.extend([(col, row2[0], row3[0], subcol) for subcol in row4[2:]])
        else:
            tuples.append((col, '', '', ''))
    
    return pd.MultiIndex.from_tuples(tuples)

def create_dataframe(headers, sample_data):
    """
    Creates a DataFrame from the processed headers and sample data.
    """
    row1, row2, row3, row4 = extract_headers(headers)
    columns = create_multi_index(row1, row2, row3, row4)
    return pd.DataFrame(sample_data, columns=columns)

def process_table(table_info, table_key):
    columns = table_info[table_key]['headers']
    table_data = table_info[table_key]['data']

    dr1 = [text.replace('\n', ' ') for text in list(table_data[0].values())]
    data_rows = [dr1]

    for i, dr in enumerate(table_data[1:], start=2):
        drs = globals()[f'dr{i}'] = [''] * 7 + list(dr.values())
        data_rows.append(drs)

    return create_dataframe(columns, data_rows)

def save_to_csv(df, filename):
    """
    Saves the DataFrame to a CSV file.
    """
    df.to_csv(filename, index=False)
    print(f"CSV file '{filename}' saved successfully.")


In [165]:
# Process international tables
table1_mm = process_table(international_table_info, 'Table_1')
save_to_csv(table1_mm, 'table1_mm.csv')

CSV file 'table1_mm.csv' saved successfully.


In [169]:
table2_mm = process_table(international_table_info, 'Table_2')
save_to_csv(table2_mm, 'table2_mm.csv')

CSV file 'table2_mm.csv' saved successfully.


In [167]:
# Process USA tables
table1_inch = process_table(usa_table_info, 'Table_1')
save_to_csv(table1_inch, 'table1_inch.csv')

CSV file 'table1_inch.csv' saved successfully.


In [168]:
# Process USA tables
table2_inch = process_table(usa_table_info, 'Table_2')
save_to_csv(table2_inch, 'table2_inch.csv')


CSV file 'table2_inch.csv' saved successfully.
